In [1]:
from os.path import join
import pandas as pd
import numpy as np

In [2]:
queries_dir = join("data", "queries")

In [3]:
cc_df = pd.read_csv(join(queries_dir, "q02_get_field_year_percentiles.csv"), index_col=0)

In [4]:
cc_df["percent_including_row"] = np.nan
cc_df["percent_excluding_row"] = np.nan
cc_df["cp_in"] = np.nan
cc_df["cp_ex"] = np.nan

In [5]:
for (source, field, year), group_df in cc_df.groupby(["source", "field", "year"]):
    total_num_papers = group_df["paper_count"].sum()
    cum_pct_in = 0.0
    cum_pct_ex = 0.0
    pct_excl_row = 0.0
    for group_row_i, row in group_df.sort_values(by="citation_count").reset_index().iterrows():
        global_row_i = row["index"]
        pct_incl_row = row["paper_count"] / total_num_papers * 100
        cum_pct_in += pct_incl_row
        
        cc_df.at[global_row_i, "percent_including_row"] = pct_incl_row
        cc_df.at[global_row_i, "percent_excluding_row"] = pct_excl_row
        cc_df.at[global_row_i, "cp_in"] = cum_pct_in
        cc_df.at[global_row_i, "cp_ex"] = cum_pct_ex
       
        cum_pct_ex += pct_incl_row
        pct_excl_row = pct_incl_row

In [6]:
cc_df.to_csv(join(queries_dir, "cumulative_frequencies.csv"))

In [7]:
cc_df.groupby(["source", "field", "year"]).sum()[["paper_count"]].reset_index().to_csv(join(queries_dir, "paper_counts.csv"))